In [62]:
import pandas as pd
import numpy as np
from openai import OpenAI
import joblib
from tqdm.notebook import tqdm
from glob import glob
import pickle
from os import path
from unicodedata import normalize

In [97]:
path_name = '/home/crazyjeannot/Documents/doctorat/AVENTURES/corpus/1000_chunks_txt/*.txt'
path_test = '/home/crazyjeannot/Documents/doctorat/AVENTURES/corpus/test_chunks_txt/*.txt'
path_chunk = '/home/crazyjeannot/Documents/doctorat/AVENTURES/corpus/all_chunks_txt/1908_Rolland-Romain_Jean-Christophe_Tome-VII-Dans-la-maison.txt_chunk_43.txt'

In [48]:
def clean_text(txt):
    txt_res = normalize("NFKD", txt).replace('\xa0', ' ')
    txt_res = txt_res.replace("\\", "").replace('\\xa0', '')
    return txt_res

In [104]:
def gpt_stuff(chunk):

    client = OpenAI(
        api_key="[OPEN-AI-KEY]",
    )

    instruction = "Give me as an output only one word, if this text is typical of the adventure novels genre; write ADVENTURE; elif unsure; write NON_ADVENTURE. Prefer NON_ADVENTURE when unsure. HELP : adventure novels are characterized by the importance of change of scenery (historical/geographical/fantastical or social) and of violent action putting the hero in mortal danger or physical peril. Typical adventure scene : someone (described as brave/heroic) doing something dangerous in a heroic manner in a wild setting."

    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": chunk}
        ],
        model="gpt-3.5-turbo-1106",
        temperature=0.2,
    )
    
    if response is not None:
        completion = response.choices[0].message.content
        return completion
    else:
        print(f'Error: Failed to get response')
        return None

In [99]:
def get_labels(path_name):
    df = pd.DataFrame(columns=['doc', 'label'])

    for doc in tqdm(glob(path_name)):
        doc_name = path.splitext(path.basename(doc))[0]
        with open(doc, encoding="utf8") as file:
            text = file.readlines()
            text_cleaned = clean_text(str(text).lower())

            label = gpt_stuff(text_cleaned)

            df = pd.concat([df, pd.DataFrame({'doc': [doc_name], 'label': [label]})])
            df_temp = df.set_index('doc')
            df_temp.to_csv('GPT_ANNOT_OUTPUT_TEMP.csv')
    df.set_index('doc', inplace=True)
    df.to_csv('GPT_ANNOT_OUTPUT_MAIN.csv')

    return df


In [100]:
df_labels = get_labels(path_name)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [102]:
df_labels

,label
doc,
1847_Dumas-Alexandre_Les-Quarante-cinq.txt_chunk_198,ADVENTURE
1843_Sue-Eugene_Les-Mysteres-de-Paris_Tome-I.txt_chunk_44,NON_ADVENTURE
1880_Boussenard_Louis_Le-tour-du-monde-dun-gamin-de-Paris.txt_chunk_108,ADVENTURE
1879_Noir_Louis_Lhomme-de-bronze.txt_chunk_125,ADVENTURE
1925_Bazin-Rene_Le-ble-qui-leve.txt_chunk_39,NON_ADVENTURE
...,...
1865_Ponson-du-Terrail-Pierre_Le-Chambrion.txt_chunk_35,ADVENTURE
1885_Amero-Constant_Le-tour-de-France-d-un-petit-Parisien.txt_chunk_84,NON_ADVENTURE
1868_Feval-Paul_Les-habits-noirs_Tome-III-La-rue-de-Jerusalem.txt_chunk_63,NON_ADVENTURE


In [103]:
df_labels.label.value_counts()

label
ADVENTURE        505
NON_ADVENTURE    495
Name: count, dtype: int64